In [ ]:
from tqdm.auto import tqdm, trange
import os
os.chdir("/home/neverix/rlhf-trojan-2024-cod")
import gadgets as gd

In [ ]:
import prompt_search
judger = prompt_search.make_judger("logprob-s-64x32x4-rt--5.0", repeat=16)
next(judger)
tokenizer = gd.tok()
options = sorted(v for p, v in tokenizer.vocab.items() if
            v < tokenizer.vocab_size
            and v not in tokenizer.all_special_ids
            and v > 2
            and "▁" not in p
            and (not any(c.isspace() for c in p))
            and all(ord(c) < 128 or c == "▁" for c in p)
            )
sudo = [tokenizer.encode(x)[-1] for x in ("SU", "DO")]
[tokenizer.decode([t]) for t in sudo]  # ensure we don't have spaces

In [ ]:
next(judger)
judger.send([sudo[0]])
judger.send([sudo[1]])
print(next(judger))